These first code blocks are for the PF coils M1 and M4.

Lucy and Dan provided info on what each column meant, we have height and width and also circuit number which represents what circuit those individual parts are apart of.

Need to do some reformatting of the original data file so it can work easily with pandas, and then we write to a parquet file. 

In [83]:
import pandas as pd
import numpy as np

def process_pfcoil_data(input_file, output_file, parquet_output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            modified_line = ','.join(line.strip().split())
            outfile.write(modified_line + '\n')

    df = pd.read_csv(output_file)

    # Rename columns and process data
    df = df.rename(columns={
        'R': 'r', 
        'Z': 'z', 
        'DeltaR': 'height', 
        'DeltaZ': 'width', 
        'circ': 'circuit_number', 
        'Name': 'name'
    })
    
    # Remove the '*' character from 'name' column
    df['name'] = df['name'].str.strip('*')
    
    # Add new column for current calculation
    df['current (mOhms)'] = df['mult'] * df['R(mOhms)']

    # Drop unnecessary columns
    df = df.drop(['N', 'mult', 'R(mOhms)'], axis=1)

    # Save DataFrame to a Parquet file
    df.to_parquet(parquet_output_file)


In [84]:
# M1
input_file = 'geometry_files/raw_files/EFIT_efit_data_pfcoils.dat_M1'
output_file = 'temp_files/pfcoils_M1.csv'
parquet_output_file = 'geometry_files/pfcoils_M1.parquet'

process_pfcoil_data(input_file, output_file, parquet_output_file)

# M4
input_file = 'geometry_files/raw_files/EFIT_efit_data_pfcoils.dat_M4'
output_file = 'temp_files/pfcoils_M4.csv'
parquet_output_file = 'geometry_files/pfcoils_M4.parquet'

process_pfcoil_data(input_file, output_file, parquet_output_file)

In [85]:
pd.read_parquet('geometry_files/pfcoils_M1.parquet', engine='fastparquet')

,r,z,height,width,ang1,ang2,circuit_number,name,current (mOhms)
0,0.1198,-1.3643,0.012,0.017,0.0,0.0,1,amc_sol_current_1,0.0
1,0.1198,-1.3463,0.012,0.017,0.0,0.0,1,amc_sol_current_2,0.0
2,0.1198,-1.3283,0.012,0.017,0.0,0.0,1,amc_sol_current_3,0.0
3,0.1198,-1.3103,0.012,0.017,0.0,0.0,1,amc_sol_current_4,0.0
4,0.1198,-1.2923,0.012,0.017,0.0,0.0,1,amc_sol_current_5,0.0
...,...,...,...,...,...,...,...,...,...
849,0.1907,0.6960,0.013,0.215,0.0,0.0,93,amm_rodgr11,6.1
850,0.1907,0.9150,0.013,0.215,0.0,0.0,94,amm_rodgr12,6.1
851,0.1944,1.1340,0.020,0.225,0.0,0.0,95,amm_rodgr13,3.7
852,0.1979,1.3500,0.028,0.225,0.0,0.0,96,amm_rodgr14,2.7


With the detectors data files, they are in a weird format with no labels. The below code blocks will reformat the data so it can be read by pandas much easier. 

Lucy provided info on what each part in the data files meant:

- For the magnetic detectors it's r, z, poloidal orientation, and I think the last one is length but need to check.
 
- For the flux loops it's r, z, 'length in terms of toroidal angle'. All FLs on MAST used in EFIT were full flux loops so the final one here is always 2pi

Writes out the parquet for MAGENTIC DETECTORS

In [86]:
import pandas as pd

def process_magnetic_detectors(input_txt_file, combined_csv_file, processed_csv_file, parquet_output_file):
    # Combine detector names with values into a CSV
    def combine_detectors(input_file, output_file):
        with open(input_file, 'r') as infile:
            lines = infile.readlines()

        combined_data = []
        # Combine detector names with their values
        for i in range(0, len(lines), 2):
            detector_name = lines[i].strip()  # Get the detector name
            if i + 1 < len(lines):  # Check if the next line exists
                values = lines[i + 1].strip()  # Get the corresponding values
                combined_data.append(f"{detector_name},{values}")  # Combine and store

        # Write combined data to a CSV file
        with open(output_file, 'w') as outfile:
            outfile.write("name,Number,r,z,poloidal_orientation,length\n")  # Write header
            for line in combined_data:
                outfile.write(line + "\n")

    # Add commas between numbers in the CSV
    def add_commas_between_numbers(input_file, output_file):
        with open(input_file, 'r') as infile:
            lines = infile.readlines()

        with open(output_file, 'w') as outfile:
            for line in lines:
                processed_line = ','.join(line.split())  # Split by whitespace and join by commas
                outfile.write(processed_line + '\n')

    # Step 1: Combine detector names and values into a CSV
    combine_detectors(input_txt_file, combined_csv_file)

    # Step 2: Add commas between numbers in the combined CSV
    add_commas_between_numbers(combined_csv_file, processed_csv_file)

    # Step 3: Process the CSV using pandas and convert it to a parquet file
    df = pd.read_csv(processed_csv_file)
    df = df.drop(['Number'], axis=1)  # Drop the 'Number' column
    df.to_parquet(parquet_output_file)  # Save as a Parquet file

# Example usage
input_txt_file = 'temp_files/magnetic_detectors_M4.txt'
combined_csv_file = 'temp_files/magnetic_detectors_M4.csv'
processed_csv_file = 'temp_files/magnetic_detectors_processed_M4.csv'
parquet_output_file = 'geometry_files/magnetic_detectors_M4.parquet'

process_magnetic_detectors(input_txt_file, combined_csv_file, processed_csv_file, parquet_output_file)

# Example usage
input_txt_file = 'temp_files/magnetic_detectors_M1.txt'
combined_csv_file = 'temp_files/magnetic_detectors_M1.csv'
processed_csv_file = 'temp_files/magnetic_detectors_processed_M1.csv'
parquet_output_file = 'geometry_files/magnetic_detectors_M1.parquet'

process_magnetic_detectors(input_txt_file, combined_csv_file, processed_csv_file, parquet_output_file)



In [91]:
pd.read_parquet('geometry_files/magnetic_detectors_M1.parquet', engine='fastparquet')

,name,r,z,poloidal_orientation,length
0,amb_ccbv01,0.1803,1.44512,90.0,0.025
1,amb_ccbv02,0.1803,1.36887,90.0,0.025
2,amb_ccbv03,0.1803,1.29262,90.0,0.025
3,amb_ccbv04,0.1803,1.21637,90.0,0.025
4,amb_ccbv05,0.1803,1.14012,90.0,0.025
...,...,...,...,...,...
89,amb_obr15,1.5913,-0.64530,0.0,0.010
90,amb_obr16,1.5913,-0.72030,0.0,0.010
91,amb_obr17,1.5913,-0.79530,0.0,0.010
92,amb_obr18,1.4401,-1.24760,0.0,0.010


Writes out for the FLUX LOOP DETECTORS (slightly different reformatting needed, so bit of repeat code)

In [87]:
import pandas as pd

def process_flux_loop_detectors(input_txt_file, combined_csv_file, processed_csv_file, parquet_output_file):
    # Step 1: Combine detector names with values into a CSV
    def combine_detectors(input_file, output_file):
        with open(input_file, 'r') as infile:
            lines = infile.readlines()

        combined_data = []
        # Combine detector names with their values
        for i in range(0, len(lines), 2):
            detector_name = lines[i].strip()  # Get the detector name
            if i + 1 < len(lines):  # Check if the next line exists
                values = lines[i + 1].strip()  # Get the corresponding values
                combined_data.append(f"{detector_name},{values}")  # Combine and store

        # Write combined data to a CSV file
        with open(output_file, 'w') as outfile:
            outfile.write("name,Number,r,z,2pi\n")  # Write header
            for line in combined_data:
                outfile.write(line + "\n")

    # Step 2: Add commas between numbers in the CSV
    def add_commas_between_numbers(input_file, output_file):
        with open(input_file, 'r') as infile:
            lines = infile.readlines()

        with open(output_file, 'w') as outfile:
            for line in lines:
                processed_line = ','.join(line.split())  # Split by whitespace and join by commas
                outfile.write(processed_line + '\n')

    # Step 1: Combine detector names and values into a CSV
    combine_detectors(input_txt_file, combined_csv_file)

    # Step 2: Add commas between numbers in the combined CSV
    add_commas_between_numbers(combined_csv_file, processed_csv_file)

    # Step 3: Process the CSV using pandas and convert it to a parquet file
    df = pd.read_csv(processed_csv_file)
    df = df.drop(['Number', '2pi'], axis=1)  # Drop the 'Number' and '2pi' columns
    df.to_parquet(parquet_output_file)  # Save as a Parquet file

# Example usage
input_txt_file = 'temp_files/flux_loop_detectors_M4.txt'
combined_csv_file = 'temp_files/flux_loop_detectors_M4.csv'
processed_csv_file = 'temp_files/flux_loop_detectors_processed_M4.csv'
parquet_output_file = 'geometry_files/flux_loop_detectors_M4.parquet'

process_flux_loop_detectors(input_txt_file, combined_csv_file, processed_csv_file, parquet_output_file)

# Example usage
input_txt_file = 'temp_files/flux_loop_detectors_M1.txt'
combined_csv_file = 'temp_files/flux_loop_detectors_M1.csv'
processed_csv_file = 'temp_files/flux_loop_detectors_processed_M1.csv'
parquet_output_file = 'geometry_files/flux_loop_detectors_M1.parquet'

process_flux_loop_detectors(input_txt_file, combined_csv_file, processed_csv_file, parquet_output_file)

In [92]:
pd.read_parquet('geometry_files/flux_loop_detectors_M1.parquet', engine='fastparquet')

,name,r,z
0,amb_fl/cc01,0.1785,1.23490
1,amb_fl/cc02,0.1785,0.92990
2,amb_fl/cc03,0.1785,0.62490
3,amb_fl/cc04,0.1785,0.31990
4,amb_fl/cc05,0.1785,0.01490
5,amb_fl/cc06,0.1785,-0.01810
6,amb_fl/cc07,0.1785,-0.32310
7,amb_fl/cc08,0.1785,-0.62810
8,amb_fl/cc09,0.1785,-0.93310
9,amb_fl/cc10,0.1785,-1.23810


This is needed to get the correct .txt file to start with

In [88]:
# Function to remove even-numbered lines from a file
def remove_even_lines(input_file, output_file):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()
    
    # Writing odd-numbered lines to the output file (1-based index)
    with open(output_file, 'w') as outfile:
        for i, line in enumerate(lines, start=1):
            if i % 2 != 0:  # Keep only odd lines
                outfile.write(line)

# Example usage
input_file = 'temp_files/flux_loop_detectors_M1.txt'
output_file = 'your_output_file.txt'
remove_even_lines(input_file, output_file)


M1 current detectors

In [89]:
import pandas as pd


def add_commas_between_numbers(input_file, output_file):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    with open(output_file, 'w') as outfile:
        for line in lines:
            # Split the line by whitespace and then join by commas
            processed_line = ','.join(line.split())
            outfile.write(processed_line + '\n')

# Example usage
input_file = 'temp_files/current_detectors_M1.csv'
output_file = 'temp_files/current_detectors_processed_M1.csv'
add_commas_between_numbers(input_file, output_file)

df = pd.read_csv(output_file)
df.rename(columns={'Detector':'name', 'Value1':'r', 'Value2': 'z', 'Angle': 'angle'}, inplace=True)
df.to_parquet("geometry_files/current_detectors_M1.parquet")

Testing parquet is fine:

In [93]:
df = pd.read_parquet("geometry_files/current_detectors_M1.parquet")
df

,name,r,z,angle
0,AMC_Plasma_Current,-,-,330.0
1,AMC_Plasma_Current,-,-,330.0
2,AMC_Plasma_Current,-,-,330.0
3,AMC_Plasma_Current,-,-,330.0
4,AMC_Plasma_Current,-,-,270.0
5,AMC_Plasma_Current,-,-,270.0
6,AMC_Plasma_Current,-,-,270.0
7,AMC_Plasma_Current,-,-,270.0
8,AMC_TF_Current,-,-,-
9,AMC_Sol_Current,-,-,-
